* Overview of integrating different Database Servers

* Create IAM Role for Redshift Federated Queries and Spectrum

* Create Postgres Database Server

* Create Postgres Database, User and Tables

```shell
psql -h retail.cfrxujtpbrro.us-east-1.rds.amazonaws.com \
  -p 5432 \
  -U postgres \
  -W
```

```sql
CREATE DATABASE retail_db;
CREATE USER retail_user WITH ENCRYPTED PASSWORD 'itversity123';
GRANT ALL ON DATABASE retail_db TO retail_user;

\q
```

```shell
psql -h retail.cfrxujtpbrro.us-east-1.rds.amazonaws.com \
  -p 5432 \
  -U retail_user \
  -d retail_db \
  -W
```

```sql
\i /Users/itversity/Research/data/retail_db_json/create_db_tables_pg.sql
\d

\q
```

* Store new Secret using Secrets Manager


**retail.password** is the secret name.

* Accessing Secrets Manager using Python boto3

In [ ]:
import boto3

In [ ]:
secret_name = "retail.secrets"
region_name = "us-east-1"

In [ ]:
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

In [ ]:
get_secret_value_response = client.get_secret_value(
    SecretId=secret_name
)

In [ ]:
get_secret_value_response

In [ ]:
get_secret_value_response['SecretString']

In [ ]:
import json

In [ ]:
secrets = json.loads(get_secret_value_response['SecretString'])

In [ ]:
secrets

* Integrating Secrets Manager with Applications

In [ ]:
import pandas as pd

In [ ]:
table_name = input('Enter table name: ')

In [ ]:
BASE_DIR = f'/Users/itversity/Research/data/retail_db_json/{table_name}'

In [ ]:
import os

In [ ]:
file_name = os.listdir(BASE_DIR)[0]

In [ ]:
file_name

In [ ]:
file_path = f'{BASE_DIR}/{file_name}'

In [ ]:
df = pd.read_json(file_path, lines=True)

In [ ]:
import boto3

secret_name = "retail.secrets"
region_name = "us-east-1"

session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

get_secret_value_response = client.get_secret_value(
    SecretId=secret_name
)

import json

s = json.loads(get_secret_value_response['SecretString'])

s

In [ ]:
get_secret_value_response = client.get_secret_value(
    SecretId=secret_name
)

In [ ]:
db_name = 'retail_db'

In [ ]:
conn = f"""postgresql://{s['username']}:{s['password']}@{s['host']}:{s['port']}/{db_name}"""

In [ ]:
df.to_sql(table_name, conn, if_exists='append', index=False)

In [1]:
import boto3
import json

def get_secrets(secret_name):
    region_name = "us-east-1"

    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    get_secret_value_response = client.get_secret_value(
        SecretId=secret_name
    )

    return json.loads(get_secret_value_response['SecretString'])

In [2]:
import os
import pandas as pd

def json_to_df(BASE_DIR, table_name):
    file_name = os.listdir(BASE_DIR.format(table_name=table_name))[0]
    file_path = f'{BASE_DIR.format(table_name=table_name)}/{file_name}'
    return pd.read_json(file_path, lines=True)

In [6]:
import sys
s = get_secrets('retail.secrets')
db_name = 'retail_db'
conn = f"""postgresql://{s['username']}:{s['password']}@{s['host']}:{s['port']}/{db_name}"""

BASE_DIR = '/Users/itversity/Research/data/retail_db_json/{table_name}'
for table_name in ['departments', 'categories', 'products', 'order_items', 'customers']:
    try:
        df = json_to_df(BASE_DIR, table_name)
        df.to_sql(table_name, conn, if_exists='append', index=False)
        print(f'{table_name} succesfully populated...')
    except Exception as err:
        print(f'{table_name} failed')
        err_type, err_obj, traceback = sys.exc_info()

        line_num = traceback.tb_lineno
        print ("\npsycopg2 ERROR:", err, "on line number:", line_num)
        print ("psycopg2 traceback:", traceback, "-- type:", err_type)

departments failed

psycopg2 ERROR: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "departments_pkey"
DETAIL:  Key (department_id)=(2) already exists.

[SQL: INSERT INTO departments (department_id, department_name) VALUES (%(department_id)s, %(department_name)s)]
[parameters: ({'department_id': 2, 'department_name': 'Fitness'}, {'department_id': 3, 'department_name': 'Footwear'}, {'department_id': 4, 'department_name': 'Apparel'}, {'department_id': 5, 'department_name': 'Golf'}, {'department_id': 6, 'department_name': 'Outdoors'}, {'department_id': 7, 'department_name': 'Fan Shop'})]
(Background on this error at: https://sqlalche.me/e/14/gkpj) on line number: 10
psycopg2 traceback: <traceback object at 0x1a29773c0> -- type: <class 'sqlalchemy.exc.IntegrityError'>
categories failed

psycopg2 ERROR: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "categories_pkey"
DETAIL:  Key (category_id)=(1) already exists.

[SQL: INSE

* Create IAM Policy for Secrets Manager

* Associate IAM Policy to IAM Role for Federated Queries and Spectrum

* Create Redshift External Schema for Postgres Database

* Run Redshift Federated Queries

* Quick recap of Glue Databases and Tables

* Associate IAM Policy to Redshift Role to grant permissions on Glue

* Create Redshift External Schema for Glue Database using Spectrum

* Run Redshift Spectrum Queries